In [129]:
from architector import view_structures, convert_io_molecule
from ase import Atoms
import architector.geometries as geo
geometry_dict = {
            f: {'vect':getattr(geo, f),'cn':len(getattr(geo, f))} for f in dir(geo) if not f.endswith("__")
        }
import pandas as pd
import architector.io_ptable as io_ptable
from scipy.spatial.distance import pdist,squareform
from ase import Atoms
import numpy as np

In [130]:
df = pd.DataFrame(geometry_dict).T

In [170]:
sdf = df[df.cn > 3]

In [171]:
sdf.vect.values[0]

[[-1.37760039, -0.62439749, 1.30856598],
 [-1.61045832, -1.1341014, -0.34675351],
 [-0.58337107, 0.69980213, -1.78043679],
 [-0.91106455, 1.77522437, -0.136161],
 [0.51673372, -0.97118193, -1.67026702],
 [1.48031468, -1.34417343, -0.04319998],
 [1.2649994, -0.26374644, 1.52650396],
 [0.50488469, 1.51472279, 1.2044527]]

In [172]:
sdf

,vect,cn
ICIGOT_geomtry,"[[-1.37760039, -0.62439749, 1.30856598], [-1.6...",8
YEMSEP_geometry,"[[1.59269395, -1.18695901, 0.23335443], [0.569...",8
am_c3_7H2O_c0,"[[-1.9903440000000001, -1.1847919999999998, -0...",7
am_c3_9H2O_c0,"[[-2.582474, -0.13784200000000002, 0.0], [1.53...",9
axial_bicapped_hexagonal_planar,"[[-1.86281576, 0.06020958, -0.72546002], [1.86...",10
axial_bicapped_trigonal_prismatic,"[[-0.77660752, 0.7455247, 1.68554848], [0.8466...",8
bicapped_trigonal_prismatic,"[[1.511, 0.0, 1.31], [-0.756, 1.309, 1.309], [...",8
capped_octahedral,"[[0.0, 0.0, 2.0], [2.0, 0.0, 0.0], [-2.0, 0.0,...",7
capped_square_antiprismatic,"[[-1, 1.0, 1.0], [1.0, 1.0, 1.0], [-1, -1.0, 1...",9
capped_trigonal_prismatic,"[[1.511, 0.0, 1.31], [-0.756, 1.309, 1.309], [...",7


In [173]:
# Step 1: Get min pairwise distance
mindists = []
vect_array = []
min_indices = []
thetas = []
# 4. Use a helper to convert the flat index into pairwise matrix indices
def unravel_condensed_index(idx, n):
    # i < j; index into condensed matrix from pdist
    # Formula from scipy docs: https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.squareform.html
    # We find i, j such that:
    # idx = n*i - i*(i+1)//2 + j - i - 1
    # Solve this iteratively:
    k = 0
    for i in range(n - 1):
        for j in range(i + 1, n):
            if k == idx:
                return i, j
            k += 1
    return None

for ind,row in sdf.iterrows():
    # 1. Compute condensed pairwise distances
    tv = np.array(row.vect)
    tv = tv / np.linalg.norm(tv,axis=1)[0]
    distances = pdist(tv, metric='euclidean')
    
    # 2. Find index of the smallest distance
    min_idx_flat = np.argmin(distances)
    mindist = distances[min_idx_flat]
    
    # 3. Convert to squareform matrix to interpret indices
    dist_matrix = squareform(distances)
    
    # 5. Get pair indices
    i, j = unravel_condensed_index(min_idx_flat, len(tv))

    mindists.append(mindist)
    min_indices.append((i,j))
    # Compute angle between them
    cos_theta = np.dot(tv[i], tv[j])
    theta = np.arccos(cos_theta)
    thetas.append(theta)
    vect_array.append(tv)
sdf.loc[:,'mindist'] = mindists
sdf['vect'] = vect_array
sdf.loc[:,'theta'] = thetas
sdf.loc[:,'min_inds'] = min_indices 

/tmp/ipykernel_778445/2425566881.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdf.loc[:,'mindist'] = mindists
/tmp/ipykernel_778445/2425566881.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdf['vect'] = vect_array
/tmp/ipykernel_778445/2425566881.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

In [174]:
sdf

,vect,cn,mindist,theta,min_inds
ICIGOT_geomtry,"[[-0.6888001963724506, -0.31219874562206335, 0...",8,0.873800,0.904299,"(0, 1)"
YEMSEP_geometry,"[[0.7963469750211621, -0.5934795050157712, 0.1...",8,0.976068,1.019672,"(2, 5)"
am_c3_7H2O_c0,"[[-0.8024822381654663, -0.4776935725284368, -0...",7,1.183963,1.264762,"(0, 6)"
am_c3_9H2O_c0,"[[-0.9985785406195072, -0.053300077056370805, ...",9,1.090296,1.191554,"(1, 4)"
axial_bicapped_hexagonal_planar,"[[-0.9314078811821249, 0.03010479003820842, -0...",10,0.884833,0.916582,"(2, 6)"
axial_bicapped_trigonal_prismatic,"[[-0.38830376015236306, 0.3727623501462649, 0....",8,0.992662,1.038735,"(4, 5)"
bicapped_trigonal_prismatic,"[[0.7555735775595968, 0.0, 0.6550637899424698]...",8,1.056477,1.117117,"(0, 7)"
capped_octahedral,"[[0.0, 0.0, 1.0], [1.0, 0.0, 0.0], [-1.0, 0.0,...",7,0.919521,0.955133,"(0, 6)"
capped_square_antiprismatic,"[[-0.5773502691896258, 0.5773502691896258, 0.5...",9,0.918858,0.956154,"(0, 8)"
capped_trigonal_prismatic,"[[0.7555735775595968, 0.0, 0.6550637899424698]...",7,1.115288,1.183082,"(1, 6)"


In [175]:
atom_types = ['Si','P','S','Ge','As','Se','Sb','Te']
covrad = {x:io_ptable.rcov1[io_ptable.elements.index(x)] for x in atom_types}

In [176]:
covrad

{'Si': 1.16,
 'P': 1.11,
 'S': 1.03,
 'Ge': 1.21,
 'As': 1.21,
 'Se': 1.16,
 'Sb': 1.4,
 'Te': 1.36}

In [26]:
# Compositions, 1 element with up to 1/2 composition of 1 other element
# For the given CN. Random assignment of positions in the cluster.
# Scale the radii such the interatomic distances of atoms would be equal to average of 
# Interatomic distances of the composition covalent radii.
# Swing charges up/down such that even number of electrons maintained (randomly)

In [177]:
sdf.iloc[0]

vect        [[-0.6888001963724506, -0.31219874562206335, 0...
cn                                                          8
mindist                                                0.8738
theta                                                0.904299
min_inds                                               (0, 1)
Name: ICIGOT_geomtry, dtype: object

In [178]:
np.random.seed(42)

In [179]:
atoms = Atoms(symbols=['C','C','C'],positions=[[0,1,2],[-1,2,3],[4,5,6]])

In [180]:
atoms.get_atomic_numbers()

array([6, 6, 6])

In [233]:
def create_scaled_clusters(refdf, sym1='S', sym2='P', sym3='Ge'):
    outrows = []
    for name,row in refdf.iterrows():
        tcn = row['cn']
        geo = row['vect']
        for n_sym2 in range(0,int(np.floor(tcn/2))):
            for n_sym3 in range(0,int(np.floor(tcn/4))):
                n_sym1 = tcn - n_sym2 - n_sym3
                avg_covrad = (covrad[sym1]*n_sym1 + covrad[sym2]*n_sym2 + covrad[sym3]*n_sym3)/tcn
                posits = geo * (avg_covrad*2) / row['theta'] * 0.95
                symbols = [sym1] * n_sym1 + [sym2] * n_sym2 + [sym3] * n_sym3
                np.random.shuffle(symbols)
                atoms = Atoms(symbols=symbols,positions=posits)
                mol = convert_io_molecule(atoms)
                charge = 0
                if (np.sum(atoms.get_atomic_numbers()) % 2) != 0:
                    charge = np.random.choice([-1,1])
                    mol.charge = charge
                mol.xtb_charge = charge
                outrow = dict()
                outrow['uid'] = name + atoms.get_chemical_formula()
                outrow['struct'] = mol.write_mol2(sym1+str(n_sym1)+sym2+str(n_sym2)+sym3+str(n_sym3), writestring=True)
                outrow['composition'] = sym1+str(n_sym1)+sym2+str(n_sym2)+sym3+str(n_sym3)
                outrow['formula'] = atoms.get_chemical_formula()
                outrow['charge'] = charge
                outrow['s1'] = sym1
                outrow['s2'] = sym2
                outrow['s3'] = sym3
                outrows.append(outrow)
    return pd.DataFrame(outrows)


In [234]:
thing = create_scaled_clusters(sdf)

In [235]:
mol = convert_io_molecule(thing.iloc[100]['struct'])
view_structures(mol)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [236]:
from tqdm import tqdm
dfs = []
for i, s1 in tqdm(enumerate(atom_types),total=len(atom_types)):
    for s2 in atom_types:
        for s3 in atom_types:
            t = create_scaled_clusters(sdf, sym1=s1, sym2=s2, sym3=s3)
            dfs.append(t)

100%|██████████| 8/8 [02:32<00:00, 19.06s/it]


In [237]:
alldf = pd.concat(dfs).reset_index()

In [248]:
remdup = alldf.drop_duplicates('uid').reset_index(drop=True).drop('index',axis=1)

In [250]:
remdup.to_csv('all_preliminary_clusters.csv',index=False)

In [223]:
from architector import CalcExecutor

In [226]:
out = CalcExecutor(alldf.iloc[0]['struct'],relax=True, save_trajectories=True)

KeyboardInterrupt: 

In [225]:
view_structures([alldf.iloc[0]['struct'],out.mol])

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [214]:
view_structures(alldf.sample(12,random_state=42).struct)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.